# 3D structure tensor - a small example

*Author: Vedrana Andersen Dahl (vand@dtu.dk), uses code by Niels Jeppesen*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vedranaa/teaching-notebooks/blob/main/Orientations_Small_3D_example.ipynb)

A small example demonstrating the use of 3D structure tensor for visualizing and clustering dominant orientation. 
We use the structure tensor implementation provided by [structure-tensor package](https://github.com/Skielex/structure-tensor). We also use [scmap module](https://github.com/vedranaa/sphere-colormap) for sperical colormap. And lastly we use small module [volvisplotyly](https://github.com/vedranaa/goodies/blob/main/volvizplotly.py) for easier 3D visualization in ploty.

In [ ]:
# Install custom-made packages and get the data.
import os
!pip install -q structure_tensor
!pip install -q scmap
!pip install -q icosphere
if not os.path.isfile('multicube.tif.py'):
    !wget 'https://data.qim.dk/demo_data/multicube.tif' -q
if not os.path.isfile('volvizplotly.py'):
    !wget 'https://raw.githubusercontent.com/vedranaa/goodies/main/volvizplotly.py' -q
if not os.path.isfile('volvizplotly_dev.py'):
    !wget 'https://raw.githubusercontent.com/vedranaa/teaching-notebooks/main/volvizplotly_dev.py' -q
if not os.path.isfile('orientation_helper.py'):
    !wget 'https://raw.githubusercontent.com/vedranaa/teaching-notebooks/main/orientation_helper.py' -q   

In [ ]:
# Import commonly used packages
import numpy as np
import matplotlib.pyplot as plt
import tifffile

# Import custom-made packages
import structure_tensor as st
import scmap
from volvizplotly_dev import volume_slicer
import orientation_helper as oh

%matplotlib inline

## Reading the data

The data is a small cube from a volumetric image og fibre composite. The cube contains five bundles in layers: UD fibre (0 deg), crossing fibre (45 deg), 90 deg fibre, -45 deg bundle, UD bundle (0 deg).

In [ ]:
vol = tifffile.imread('multicube.tif')
# Transpose (rotate), just for better illustration
vol = np.transpose(vol, (1, 0, 2))

print(f'Volume shape is: {vol.shape}')
print(f'Volume data type is: {vol.dtype}')
print(f'Intensity range is: {vol.min():03}-{vol.max()}')

# Visualize volume slices and histogram
fig, ax = plt.subplots(1, 4, figsize=(15,5))
ax[0].imshow(vol[vol.shape[0]//2], cmap=plt.cm.gray)
ax[1].imshow(vol[:, vol.shape[1]//2, :], cmap=plt.cm.gray)
ax[2].imshow(vol[:, :, vol.shape[2]//2], cmap=plt.cm.gray)
ax[3].hist(vol.ravel(), np.arange(257)-0.5)
plt.show()

We visualize the orthogonal cross sections and the surface of 3D volume in plotly.

In [ ]:
volume_slicer(vol, ['mid', 'mid', 'mid'])

In [ ]:
volume_slicer(vol, ['ends', 'ends', 'ends'])

## Computing the structure tensor and the dominant orientation
Computation of structure tensor requires only two parameters: the noise scale sigma and the integration scale rho. Parameter sigma controls smothing while computing gradientes, and structures smaller than sigma will be removed by smoothing. Parameter rho gives the size over the neighborhood in which the orientation is to be analysed for every volume voxel. Larger rho will result in a smoother orientation field.

Structure tensor is a 3x3 matrix, but as it is symmetrical we only carry values of 6 elements: $s_{xx}$, $s_{yy}$, $s_{zz}$, $s_{xy}$, $s_{xz}$ and $s_{yz}$.

Eigenvalues (val) carry the information about the degree of anisotropy - this is not used or visualized here. Eigenvectors (vec) carry the orientation information, as $x$, $y$, and $z$ component of the orientation vector.

In [ ]:
# Change data type to double
vol = vol.astype(float)/255

In [ ]:
sigma = 0.5 # noise scale
rho = 2 # integration scale
S = st.structure_tensor_3d(vol, sigma, rho)
val, vec = st.eig_special_3d(S)

print(f'The volume has a shape {vol.shape}, i.e. {vol.size} voxels.')
print(f'Structure tensor information is carried in a {S.shape} array.')
print(f'Orientation information is carried in a {vec.shape} array.')

## Visualizing the dominant orientation
Here we investigate only dominant orientation, ignoring shape measures. 

Visualizing subset of dominant orientetions as points on a unit sphere. I choose fan coloring (`Duo` in `scmap`) which uses hsv colors for orientatins in a certain plane, here a $xy$ plane, and gray color for the orientations orthogonal to this plane. This coloring is convenient since all fibre bundles lay in $xy$ plane.


In [ ]:
# theta = 90
# st = np.sin(theta*np.pi/180)
# ct = np.cos(theta*np.pi/180)
# rotation = np.array([[ct, -st, 0], [st, ct, 0], [0, 0, 1]])
# reflection = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 1]])
# coloring = scmap.Duo(rotation=reflection@rotation)
coloring=scmap.Duo()
oh.show_coloring_sphere(coloring)

Visualizing subset of dominant orientetions as points on a unit sphere.

In [ ]:
random_subset = 1000
oh.show_scatter_points(vec, coloring, random_subset)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15, 10))
for z, a in zip([30, 60, 85, 123], ax):
  oh.show_vol_flow(vol, vec, z=z, s=10, double_arrow = True, ax=a) 
plt.show()

When visualizing the orientation as color, I choose fan coloring (`Duo` in `scmap`) which uses hsv colors for orientatins in a certain plane, here a $xy$ plane, and gray color for the orientations orthogonal to this plane. This coloring is convenient since all fibre bundles lay in $xy$ plane.

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15, 10))
for z, a in zip([30, 60, 85, 123], ax):
  oh.show_vol_orientation(vol, vec, z=z, coloring=coloring, ax=a)
plt.show()

In [ ]:
# The support of volume slicer for rgb images is experimental
volrgb = coloring(vec.reshape(3, -1).T).reshape((vec.shape[1:] + (3,)))
volume_slicer(volrgb, ['mid', 'mid', 'mid'])